In [591]:
#import
import time
import pandas as pd
import numpy as np
#import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium .webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains


In [595]:
class StocksScraper:
    def __init__(self,driver,timeout=10):
        self.driver=driver
        self.wait= WebDriverWait(self.driver,timeout=timeout)
        self.data=[]

    def wait_for_page_to_load(self):
    
        page_title=self.driver.title
        try:
            self.wait.until(
                lambda d: d.execute_script("return document.readyState") == 'complete'
            )
        except:
            
            print(f" The page {page_title} did not fully loaded within the given duration.")
        else:
            
            print(f"The page \"{page_title}\" is fully loaded.")


    

    def access_url(self,url):
        self.driver.get(url)
        self.wait_for_page_to_load()


    def access_most_active_stocks(self):
        #hovering on market menu 
        actions =ActionChains(self.driver)# it will help to clicking the mouse and moving thst called hovering 
        market_menu =self.wait.until(
            EC.presence_of_element_located((By.XPATH, '/html[1]/body[1]/div[2]/header[1]/div[1]/div[1]/div[1]/div[4]/div[1]/div[1]/ul[1]/li[3]/a[1]/span[1]'))
        )#we get this data from (we get this data from selector hum(abs XPATH))
        actions.move_to_element(market_menu).perform()
        
        #trending ticker
        trending_ticker=self.wait.until(
            EC.element_to_be_clickable((By.XPATH,'/html[1]/body[1]/div[2]/header[1]/div[1]/div[1]/div[1]/div[4]/div[1]/div[1]/ul[1]/li[3]/div[1]/ul[1]/li[4]/a[1]/div[1]'))
        )#we get this data from (we get this data from selector hum(abs XPATH))
        trending_ticker.click()
        self.wait_for_page_to_load()
        
        #click on Most Active 
        most_active=self.wait.until(
            EC.element_to_be_clickable((By.XPATH,'/html[1]/body[1]/div[2]/main[1]/section[1]/section[1]/section[1]/article[1]/section[1]/div[1]/nav[1]/ul[1]/li[1]/a[1]/span[1]'))
        )#we get this data from (we get this data from selector hum(abs XPATH))
        
        most_active.click()
        self.wait_for_page_to_load()


    def extract_stocks_data(self):
        ##navigating tags
        #scraping the data
        while True:
               #scraping 
            self.wait.until(
                EC.presence_of_element_located((By.TAG_NAME,"table"))
            )
            rows=self.driver.find_elements(By.CSS_SELECTOR,"table tbody tr")
            for row in rows:
                values=row.find_elements(By.TAG_NAME,"td")
                [val.text for val in  values]
                stock={
                    "Symbol":values[0].text,
                    "Name":values[1].text,
                    "Price":values[3].text,
                    "Change":values[4].text,
                    "Change %":values[5].text,
                    "Volume":values[6].text,
                    "Avg_Vol_3m":values[7].text,
                    "Market_Cap":values[8].text,
                    "PE_Ratio":values[9].text,
                    '52_WK_Change %':values[10].text
                    
                }
                self.data.append(stock)
                #break
            #break
                
            
        
            #click next 
            try:
                next_button=self.wait.until(
                    EC.element_to_be_clickable((By.XPATH,'//*[@id="nimbus-app"]/section/section/section/article/section[1]/div/div[3]/div[3]/button[3]'))
                )
            except:
                print("the \"next\" botton is not clickable. we have navigate through all the pages.")
                break
        
            else:
                next_button.click()
                time.sleep(2)
        

    def clean_and_save_data(self,filename='temp'):
        stocks_df=(
            pd
            .DataFrame(self.data)
            .apply(lambda col: col.str.strip() if col.dtype=="object" else col)
            .assign(
                Price=lambda df_: pd.to_numeric(df_.Price),
                Avg_Vol_3m=lambda df_: pd.to_numeric(df_.Avg_Vol_3m.str.replace("M","").str.replace(",", "")),
                Change=lambda df_: pd.to_numeric(df_.Change.str.replace("+","")),
                Volume=lambda df_: pd.to_numeric(df_.Volume.str.replace("M","")),
                Market_Cap=lambda df_: df_.Market_Cap.str.replace(",", "").apply(
                    lambda val: (
                        float(val.replace("B", ""))
                        if "B" in val else
                        float(val.replace("T", "")) * 1000
                        if "T" in val else
                        float(val.replace("M", "")) / 1000
                        if "M" in val else
                        np.nan  # fallback for '-' or unexpected format
                    )
                ),
        
                PE_Ratio=lambda df_:(
                    df_
                    .PE_Ratio
                    .replace("-",np.nan)
                    .str.replace(",","")
                    .pipe(lambda col:pd.to_numeric(col))
                )
            )
            .rename(columns={
                
                "Price": "Price(USD)",
                "Volume":"Volume_M",
                "Market_Cap":"Market_Cap_B",
                
            })
            
            
        )
        stocks_df["Change %"] = stocks_df["Change %"].str.replace("%", "").astype(float)
        stocks_df["52_WK_Change %"] = stocks_df["52_WK_Change %"].str.replace("%", "").astype(float)
        

        stocks_df.to_csv(f"{filename}.csv", index=False)



        













        

In [597]:
if __name__=="__main__":
    driver=webdriver.Chrome()
    driver.maximize_window()

    url='https://finance.yahoo.com'
    scraper=StocksScraper(driver, 5)

    scraper.access_url(url)
    scraper.access_most_active_stocks()
    scraper.extract_stocks_data()
    scraper.clean_and_save_data("yahoo_finance_stocks")

    driver.quit()


The page "Yahoo Finance - Stock Market Live, Quotes, Business & Finance News" is fully loaded.
The page "Yahoo Finance - Stock Market Live, Quotes, Business & Finance News" is fully loaded.
The page "Top Trending Stocks: US stocks with the highest interest today - Yahoo Finance" is fully loaded.
the "next" botton is not clickable. we have navigate through all the pages.


In [402]:

url='https://finance.yahoo.com'

driver=webdriver.Chrome()
driver.maximize_window()

wait=WebDriverWait(driver,5)
def wait_for_page_to_load(driver, wait):
    
    page_title=driver.title
    try:
        wait.until(
            lambda d: d.execute_script("return document.readyState") == 'complete'
        )
    except:
        
        print(f" The page {page_title} did not fully loaded within the given duration.")
    else:
        
        print(f"The page \"{page_title}\" is fully loaded.")

        

driver.get(url)
wait_for_page_to_load(driver, wait)

#hovering on market menu 
actions =ActionChains(driver)# it will help to clicking the mouse and moving thst called hovering 
market_menu =wait.until(
    EC.presence_of_element_located((By.XPATH, '/html[1]/body[1]/div[2]/header[1]/div[1]/div[1]/div[1]/div[4]/div[1]/div[1]/ul[1]/li[3]/a[1]/span[1]'))
)#we get this data from (we get this data from selector hum(abs XPATH))
actions.move_to_element(market_menu).perform()

#trending ticker
trending_ticker=wait.until(
    EC.element_to_be_clickable((By.XPATH,'/html[1]/body[1]/div[2]/header[1]/div[1]/div[1]/div[1]/div[4]/div[1]/div[1]/ul[1]/li[3]/div[1]/ul[1]/li[4]/a[1]/div[1]'))
)#we get this data from (we get this data from selector hum(abs XPATH))
trending_ticker.click()
wait_for_page_to_load(driver, wait)

#click on Most Active 
most_active=wait.until(
    EC.element_to_be_clickable((By.XPATH,'/html[1]/body[1]/div[2]/main[1]/section[1]/section[1]/section[1]/article[1]/section[1]/div[1]/nav[1]/ul[1]/li[1]/a[1]/span[1]'))
)#we get this data from (we get this data from selector hum(abs XPATH))

most_active.click()
wait_for_page_to_load(driver, wait)



data=[]
##navigating tags
#scraping the data
while True:
    
    #scraping 
    wait.until(
        EC.presence_of_element_located((By.TAG_NAME,"table"))
    )
    rows=driver.find_elements(By.CSS_SELECTOR,"table tbody tr")
    print(len(rows))
    for row in rows:
        values=row.find_elements(By.TAG_NAME,"td")
        print([val.text for val in  values])
        stock={
            "Symbol":values[0].text,
            "Name":values[1].text,
            "Price":values[3].text,
            "Change":values[4].text,
            "Change %":values[5].text,
            "Volume":values[6].text,
            "Avg_Vol_3m":values[7].text,
            "Market_Cap":values[8].text,
            "PE_Ratio":values[9].text,
            '52_WK_Change %':values[10].text
            
        }
        data.append(stock)
        #break
    #break
        
    

    #click next 
    try:
        next_button=wait.until(
            EC.element_to_be_clickable((By.XPATH,'//*[@id="nimbus-app"]/section/section/section/article/section[1]/div/div[3]/div[3]/button[3]'))
        )
    except:
        print("the \"next\" botton is not clickable. we have navigate through all the pages.")
        break

    else:
        next_button.click()
        time.sleep(2)

driver.quit()


The page "Yahoo Finance - Stock Market Live, Quotes, Business & Finance News" is fully loaded.
The page "Yahoo Finance - Stock Market Live, Quotes, Business & Finance News" is fully loaded.
The page "Top Trending Stocks: US stocks with the highest interest today - Yahoo Finance" is fully loaded.
25
['2222.SR', 'Saudi Arabian Oil Company', '', '24.92', '-1.38', '-5.25%', '24.031M', '9.194M', '6.027T', '32.08', '8.24%', '']
['INTC', 'Intel Corporation', '', '19.85', '-2.58', '-11.50%', '174.451M', '104.094M', '86.558B', '-', '-47.74%', '']
['F', 'Ford Motor Company', '', '9.58', '+0.04', '+0.42%', '170.516M', '106.236M', '38.095B', '6.56', '-28.56%', '']
['TSLA', 'Tesla, Inc.', '', '239.43', '-27.85', '-10.42%', '179.252M', '101.586M', '770.131B', '117.37', '38.41%', '']
['PLTR', 'Palantir Technologies Inc.', '', '74.01', '-9.59', '-11.47%', '145.653M', '100.208M', '173.58B', '389.53', '221.78%', '']
['AMZN', 'Amazon.com, Inc.', '', '171.00', '-7.41', '-4.15%', '118.862M', '43.002M', '1.

In [420]:
a=pd.DataFrame(data)
a.head(20)

,Symbol,Name,Price,Change,Change %,Volume,Avg_Vol_3m,Market_Cap,PE_Ratio,52_WK_Change %
0,NVDA,NVIDIA Corporation,94.31,-7.49,-7.36%,525.608M,286.927M,2.301T,32.08,8.24%
1,INTC,Intel Corporation,19.85,-2.58,-11.50%,174.451M,104.094M,86.558B,-,-47.74%
2,F,Ford Motor Company,9.58,+0.04,+0.42%,170.516M,106.236M,38.095B,6.56,-28.56%
3,TSLA,"Tesla, Inc.",239.43,-27.85,-10.42%,179.252M,101.586M,770.131B,117.37,38.41%
4,PLTR,Palantir Technologies Inc.,74.01,-9.59,-11.47%,145.653M,100.208M,173.58B,389.53,221.78%
5,AMZN,"Amazon.com, Inc.",171.00,-7.41,-4.15%,118.862M,43.002M,1.812T,30.98,-7.66%
6,AAPL,Apple Inc.,188.38,-14.81,-7.29%,124.922M,54.723M,2.83T,29.95,11.83%
7,AAL,American Airlines Group Inc.,9.46,-0.05,-0.47%,109.795M,48.623M,6.221B,7.63,-31.94%
8,LCID,"Lucid Group, Inc.",2.2800,-0.0400,-1.72%,105.732M,92.384M,6.912B,-,-13.64%
9,BAC,Bank of America Corporation,34.39,-2.83,-7.60%,95.717M,41.37M,261.46B,10.71,-8.29%


In [553]:
stocks_df=(
    pd
    .DataFrame(data)
    .apply(lambda col: col.str.strip() if col.dtype=="object" else col)
    .assign(
        Price=lambda df_: pd.to_numeric(df_.Price),
        Avg_Vol_3m=lambda df_: pd.to_numeric(df_.Avg_Vol_3m.str.replace("M","").str.replace(",", "")),
        Change=lambda df_: pd.to_numeric(df_.Change.str.replace("+","")),
        Volume=lambda df_: pd.to_numeric(df_.Volume.str.replace("M","")),
        Market_Cap=lambda df_: df_.Market_Cap.str.replace(",", "").apply(
            lambda val: (
                float(val.replace("B", ""))
                if "B" in val else
                float(val.replace("T", "")) * 1000
                if "T" in val else
                float(val.replace("M", "")) / 1000
                if "M" in val else
                np.nan  # fallback for '-' or unexpected format
            )
        ),

        PE_Ratio=lambda df_:(
            df_
            .PE_Ratio
            .replace("-",np.nan)
            .str.replace(",","")
            .pipe(lambda col:pd.to_numeric(col))
        )
    )
    .rename(columns={
        
        "Price": "Price(USD)",
        "Volume":"Volume_M",
        "Market_Cap":"Market_Cap_B",
        
    })
    
    
)
stocks_df["Change %"] = stocks_df["Change %"].str.replace("%", "").astype(float)
stocks_df["52_WK_Change %"] = stocks_df["52_WK_Change %"].str.replace("%", "").astype(float)



In [555]:
stocks_df.dtypes

Symbol             object
Name               object
Price(USD)        float64
Change            float64
Change %          float64
Volume_M          float64
Avg_Vol_3m        float64
Market_Cap_B      float64
PE_Ratio          float64
52_WK_Change %    float64
dtype: object

In [551]:
stocks_df.head()

,Symbol,Name,Price(USD),Change,Change %,Volume_M,Avg_Vol_3m,Market_Cap_B,PE_Ratio,52_WK_Change %
0,NVDA,NVIDIA Corporation,94.31,-7.49,-7.36,525.608,286.927,2301.000,32.08,8.24
1,INTC,Intel Corporation,19.85,-2.58,-11.50,174.451,104.094,86.558,NaN,-47.74
2,F,Ford Motor Company,9.58,0.04,0.42,170.516,106.236,38.095,6.56,-28.56
3,TSLA,"Tesla, Inc.",239.43,-27.85,-10.42,179.252,101.586,770.131,117.37,38.41
4,PLTR,Palantir Technologies Inc.,74.01,-9.59,-11.47,145.653,100.208,173.580,389.53,221.78


In [523]:
stocks_df.to_csv("stocks.csv", index=False)

In [525]:
pd.read_csv("stocks.csv")

,Symbol,Name,Price(USD),Change,Change %,Volume_M,Avg_Vol_3m,Market_Cap_B,PE_Ratio,52_WK_Change %
0,NVDA,NVIDIA Corporation,94.31,-7.49,-7.36,525.608,286.927,2301.000,32.08,8.24
1,INTC,Intel Corporation,19.85,-2.58,-11.50,174.451,104.094,86.558,NaN,-47.74
2,F,Ford Motor Company,9.58,0.04,0.42,170.516,106.236,38.095,6.56,-28.56
3,TSLA,"Tesla, Inc.",239.43,-27.85,-10.42,179.252,101.586,770.131,117.37,38.41
4,PLTR,Palantir Technologies Inc.,74.01,-9.59,-11.47,145.653,100.208,173.580,389.53,221.78
...,...,...,...,...,...,...,...,...,...,...
524,PAGP,"Plains GP Holdings, L.P.",18.66,-1.99,-9.64,5.289,1.757,4.350,35.88,-5.04
525,DOC,"Healthpeak Properties, Inc.",18.43,-1.03,-5.29,5.037,4.824,12.875,51.19,-2.90
526,K,Kellanova,82.24,-0.31,-0.38,5.006,2.438,28.391,21.20,43.08
527,OGN,Organon & Co.,13.05,-1.40,-9.69,5.003,3.355,3.366,3.92,-28.14
